In [28]:
import time
import numpy as np
import cupy

In [30]:
def print_nice(statement):
    print(50 * "-")
    print(statement)
    print(50 * "-")

In [31]:
# Run benchmark
MSIZE = 7500
print_nice(f"Benchmark Matrix Multiplication: Size {MSIZE} x {MSIZE}")
np.random.seed(42)
A_np = np.random.rand(MSIZE, MSIZE).astype(np.float32)
B_np = np.random.rand(MSIZE, MSIZE).astype(np.float32)

print("Starting NumPy (CPU) computation ...")
start_time_np = time.time()
C_np = np.dot(A_np, B_np)

end_time_np = time.time()
cpu_time = end_time_np - start_time_np
print(f"NumPy (CPU) Time: {cpu_time:.4f} seconds")

--------------------------------------------------
Benchmark Matrix Multiplication: Size 7500 x 7500
--------------------------------------------------
Starting NumPy (CPU) computation ...
NumPy (CPU) Time: 1.3944 seconds


In [32]:
# Phase A: Data transfer (CPU --> GPU)
start_time_transfer = time.time()
A_cp = cupy.asarray(A_np)
B_cp = cupy.asarray(B_np)

cupy.cuda.Stream.null.synchronize()
end_time_transfer = time.time()
transfer_time = end_time_transfer - start_time_transfer
print(f"Phase A. H2D Transfer Time: {transfer_time:.4f} seconds")

print("Starting CuPy (GPU) computation")
start_time_compute = time.time()

C_cp = cupy.dot(A_cp, B_cp)
# Wait for the GPU to finish all operations (essential for accurate timing)
cupy.cuda.Stream.null.synchronize()

end_time_compute = time.time()
compute_time = end_time_compute - start_time_compute

# Total GPU elapsed time (Transfer + Compute)
total_gpu_time = transfer_time + compute_time

is_correct = np.allclose(C_np[0,0], cupy.asnumpy(C_cp)[0,0])
print(f"CuPy (GPU) Time: {compute_time:.4f} seconds")
print(f"Results match (first element): {is_correct}")

if compute_time > 0:
    pure_speedup = cpu_time / compute_time
    print(f"\n--- Pure GPU Speedup (CPU/Compute): {pure_speedup:.1f} x faster ---")
if total_gpu_time > 0:
    total_speedup = cpu_time / total_gpu_time
    print(f"\n--- Total Speedup (CPU/Total GPU): {total_speedup:.1f} x faster ---")

AttributeError: module 'cupy' has no attribute 'asarray'

In [ ]:
A_cp = cp.asarray(A_np)
B_cp = cp.asarray(B_np)

print("Starting CuPy (GPU) computation")
start_time_cp = time.time()

C_cp = cp.dot(A_cp, B_cp)
# Wait for the GPU to finish all operations (essential for accurate timing)
cp.cuda.Stream.null.synchronize()

end_time_cp = time.time()
gpu_time = end_time_cp - start_time_cp

is_correct = np.allclose(C_np[0,0], cp.asnumpy(C_cp)[0,0])
print(f"CuPy (GPU) Time: {gpu_time:.4f} seconds")
print(f"Results match (first element): {is_correct}")

if gpu_time > 0:
    speedup = cpu_time / gpu_time
    print(f"\n---GPU Speedup: {speedup:.1f} x faster ---")

AttributeError: module 'cupy' has no attribute 'asarray'